In [1]:
import os
from dotenv import load_dotenv

import pandas as pd

from shroomdk import ShroomDK

In [2]:
load_dotenv()
# Initialize `ShroomDK` with your API Key
sdk = ShroomDK(os.getenv("FLIPSIDE_API_KEY"))

In [16]:
def get_aave_price(sdk: ShroomDK) -> float:
    sql = f"""
        SELECT * 
        FROM ethereum.core.fact_hourly_token_prices
        WHERE SYMBOL ILIKE 'aave'
            AND hour::DATE = CURRENT_DATE
        ORDER BY hour DESC
        LIMIT 1;
        """

    query_result_set = sdk.query(sql)
    return round(query_result_set.records[0]["price"], 2)

def get_user_top_lending_pools(sdk: ShroomDK, user_id: str, time_interval: int) -> pd.DataFrame:
    sql = f"""
        SELECT block_timestamp::DATE AS date,
            symbol,
            SUM(supplied_usd) AS supplied_usd
        FROM flipside_prod_db.aave.deposits
        WHERE depositor_address ILIKE '{user_id}'
            AND block_timestamp::DATE >= CURRENT_DATE - {time_interval}
            AND symbol IS NOT NULL
        GROUP BY 1, 2;
        """
    results = sdk.query(sql)
    return pd.DataFrame(results.records)


def get_hourly_withdrawals_today(sdk: ShroomDK) -> pd.DataFrame:
    sql = f"""
        SELECT HOUR(block_timestamp) AS Hour,
            COUNT(DISTINCT(tx_id)) AS "Total Transactions",
            ROUND(SUM(withdrawn_usd)) AS "Total Withdrawn in USD"
        FROM flipside_prod_db.aave.withdraws
        WHERE block_timestamp::DATE = (SELECT MAX(block_timestamp::DATE) FROM flipside_prod_db.aave.withdraws)
            AND symbol IS NOT NULL
        GROUP BY 1
        ORDER BY 1;
        """
    results = sdk.query(sql)
    return pd.DataFrame(results.records)

In [4]:
get_aave_price(sdk)

103.08

In [28]:
def get_user_latest_borrowing_activity(sdk: ShroomDK, user_id: str, time_interval: int) -> pd.DataFrame:
    sql = f"""
        SELECT block_timestamp::DATE AS date,
            symbol,
            borrowed_usd
        FROM flipside_prod_db.aave.borrows
        WHERE borrower_address ILIKE '{user_id}'
            AND block_timestamp::DATE >= CURRENT_DATE - {time_interval}
        ORDER BY 1 DESC 
        LIMIT 25;
        """
    results = sdk.query(sql)
    return pd.DataFrame(results.records)

def get_user_latest_borrowing_activity(sdk: ShroomDK, user_id: str, time_interval: int) -> pd.DataFrame:
    sql = f"""
        SELECT block_timestamp::DATE AS date,
            symbol,
            borrowed_usd
        FROM flipside_prod_db.aave.borrows
        WHERE borrower_address ILIKE '{user_id}'
            AND block_timestamp::DATE >= CURRENT_DATE - {time_interval}
        ORDER BY 1 DESC 
        LIMIT 25;
        """
    results = sdk.query(sql)
    return pd.DataFrame(results.records)

In [32]:
get_user_latest_borrowing_activity(sdk, "0x02e1ee540a03b3524c38dd2e298184795dedda32", 13)

,date,symbol,borrowed_usd
0,2022-08-07,WETH,34489.031478
1,2022-08-07,WETH,5097.647092
2,2022-07-30,WETH,139197.439751


In [ ]:
0xc496a9b328dd69a0c335874f84b70225a23fb2a5
0x794f2331f69f9d276a3a006953669cd2fc23ab92